In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer

In [2]:
cancer = load_breast_cancer()
data = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data['Class'] = cancer.target

X = data.drop('Class', axis=1)
y = data['Class']

x_train, x_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=3,
                                                    stratify=y)

x_train_bin = x_train.apply(pd.cut, bins=2, labels=[1, 0]).values
x_test_bin = x_test.apply(pd.cut, bins=2, labels=[1, 0]).values

In [3]:
class MPNeuron:
    def __init__(self):
        self.b = 0

    def Model(self, x):
        return np.sum(x) >= self.b

    def fit(self, x, y):
        accuracy = {}
        for b in range(x.shape[1] + 1):
            self.b = b
            yhat = []
            for row in x:
                yhat.append(self.Model(row))

            accuracy[b] = accuracy_score(y, yhat)
        
        best_b = max(accuracy, key=accuracy.get)
        self.b = best_b
        return accuracy, best_b, accuracy[best_b]

    def predict(self, x, y):
        yhat = []
        for row in x:
            yhat.append(self.Model(row))

        accuracy = accuracy_score(y, yhat)
        return accuracy

In [4]:
neuron = MPNeuron()
accuracy, best_b, accuracy_model = neuron.fit(x_train_bin, y_train)
print(f"Training set accuracy: {(accuracy_model * 100):.2f}%")

accuracy = neuron.predict(x_test_bin, y_test)
print(f"Testing set accuracy: {(accuracy * 100):.2f}%")

Training set accuracy: 84.84%
Testing set accuracy: 87.72%


In [5]:
class Perceptron:
    def __init__(self):
        self.w = None
        self.b = None

    def model(self, x):
        return 1 if np.dot(self.w, x) >= self.b else 0

    def predict(self, X):
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(result)
        return np.array(Y)

    def fit(self, X, Y, epochs=1, lr=1):
        self.w = np.ones(X.shape[1])
        self.b = 0

        accuracy = {}
        max_accuracy = 0
        for i in range(epochs):
            for x, y in zip(X, Y):
                y_pred = self.model(x)
                if y == 1 and y_pred == 0:
                    self.w = self.w + lr * x
                    self.b = self.b - lr * 1
                elif y == 0 and y_pred == 1:
                    self.w = self.w - lr * x
                    self.b = self.b + lr * 1
            
            accuracy[i] = accuracy_score(self.predict(X), Y)

            if (accuracy[i] > max_accuracy):
                max_accuracy = accuracy[i]
                chkptw = self.w
                chkptb = self.b
        
        self.w = chkptw
        self.b = chkptb

In [6]:
model = Perceptron()
model.fit(x_train.values, y_train.values, 10000, 0.0001)

yhat = model.predict(x_train.values)
accuracy = accuracy_score(y_train.values, yhat)
print(f"Training set accuracy: {(accuracy * 100):.2f}%")

yhat = model.predict(x_test.values)
accuracy = accuracy_score(y_test.values, yhat)
print(f"Testing set accuracy: {(accuracy * 100):.2f}%")

Training set accuracy: 93.85%
Testing set accuracy: 94.74%
